In [1]:
# !pip install scikit-learn-intelex

In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import pandas as pd
import numpy as np
import joblib
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
# import plotly.offline as pyo

import plotly.io as pio
pio.renderers.default='notebook'

import matplotlib.pyplot as plt
# pyo.init_notebook_mode()

import statsmodels.graphics.tsaplots as tsaplots

import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.svm import SVR

from time import time

In [3]:
# import scipy
# scipy.test()

In [3]:
X_train = pd.read_csv('C:/Users/vmore/notebooks/TFM/Modelos eolica/datos_exogena/X_train_2y.csv', parse_dates=[0], index_col='datetime_utc')
y_train = pd.read_csv('C:/Users/vmore/notebooks/TFM/Modelos eolica/datos_exogena/y_train_2y.csv', parse_dates=[0], index_col='datetime_utc')
X_test = pd.read_csv('C:/Users/vmore/notebooks/TFM/Modelos eolica/datos_exogena/X_test.csv', parse_dates=[0], index_col='datetime_utc')
y_test = pd.read_csv('C:/Users/vmore/notebooks/TFM/Modelos eolica/datos_exogena/y_test.csv', parse_dates=[0], index_col='datetime_utc')
test_data = pd.read_csv('C:/Users/vmore/notebooks/TFM/Modelos eolica/datos_exogena/test_data.csv', parse_dates=[0], index_col='datetime_utc')

# t+1 (2)

In [4]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 12)
l_epsilon = [2.**k for k in range(-6, 0)]

In [5]:
l_C

[10.0, 100.0, 1000.0, 10000.0, 100000.0]

In [6]:
l_gamma

[0.0013020833333333333,
 0.0026041666666666665,
 0.005208333333333333,
 0.010416666666666666]

In [7]:
l_epsilon

[0.015625, 0.03125, 0.0625, 0.125, 0.25, 0.5]

In [8]:
start = time()
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t1 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t1.fit(X_train.values, y_train['percentage(t+1)'].values)

training_time = time() - start
print(training_time/60, ' minutos')

132.82574286460877  minutos


In [9]:
best_svr_score_t1 = grid_search_SVM_t1.best_score_
best_svr_model_t1 = grid_search_SVM_t1.best_estimator_

In [10]:
best_svr_model_t1.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=100.0, epsilon=0.015625, gamma=0.005208333333333333))])

In [11]:
print(best_svr_model_t1.regressor_)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=100.0, epsilon=0.015625, gamma=0.005208333333333333))])


In [12]:
y_pred_svr = best_svr_model_t1.predict(X_test.values)

In [13]:
MAE_SVR_t1=metrics.mean_absolute_error(y_test['percentage(t+1)'], y_pred_svr)
print('MAE en t+1: {:2.2%}'.format(MAE_SVR_t1))

MAE en t+1: 0.76%


In [14]:
joblib.dump(best_svr_model_t1, "modelos_exogena/SVR_t1_model.joblib")

['modelos_exogena/SVR_t1_model.joblib']

# t+1

In [4]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 30)
l_epsilon = [2.**k for k in range(-6, 0)]

In [5]:
l_C

[10.0, 100.0, 1000.0, 10000.0, 100000.0]

In [6]:
l_gamma

[0.0005208333333333333,
 0.0010416666666666667,
 0.0020833333333333333,
 0.004166666666666667]

In [7]:
l_epsilon

[0.015625, 0.03125, 0.0625, 0.125, 0.25, 0.5]

In [8]:
start = time()
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t1 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t1.fit(X_train.values, y_train['percentage(t+1)'].values)

training_time = time() - start
print(training_time/60, ' minutos')

105.47432128190994  minutos


In [9]:
best_svr_score_t1 = grid_search_SVM_t1.best_score_
best_svr_model_t1 = grid_search_SVM_t1.best_estimator_

In [10]:
best_svr_model_t1.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=1000.0, epsilon=0.0625, gamma=0.0010416666666666667))])

In [14]:
print(best_svr_model_t1.regressor_)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=1000.0, epsilon=0.0625, gamma=0.0010416666666666667))])


In [11]:
y_pred_svr = best_svr_model_t1.predict(X_test.values)

In [12]:
MAE_SVR_t1=metrics.mean_absolute_error(y_test['percentage(t+1)'], y_pred_svr)
print('MAE en t+1: {:2.2%}'.format(MAE_SVR_t1))

MAE en t+1: 1.36%


In [13]:
joblib.dump(best_svr_model_t1, "modelos_exogena/SVR_t1_model.joblib")

['modelos_exogena/SVR_t1_model.joblib']

# t+2 (2)

In [4]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-7, -3)]) / 30)
l_epsilon = [2.**k for k in range(-6, 0)]

In [5]:
l_C

[10.0, 100.0, 1000.0, 10000.0, 100000.0]

In [6]:
l_gamma

[0.00026041666666666666,
 0.0005208333333333333,
 0.0010416666666666667,
 0.0020833333333333333]

In [7]:
l_epsilon

[0.015625, 0.03125, 0.0625, 0.125, 0.25, 0.5]

In [8]:
start = time()
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t2 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t2.fit(X_train.values, y_train['percentage(t+2)'].values)

training_time = time() - start
print(training_time/60, ' minutos')

23.911758430798848  minutos


In [9]:
best_svr_score_t2 = grid_search_SVM_t2.best_score_
best_svr_model_t2 = grid_search_SVM_t2.best_estimator_

In [10]:
best_svr_model_t2.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=100.0, epsilon=0.03125, gamma=0.0010416666666666667))])

In [11]:
print(best_svr_model_t2.regressor_)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=100.0, epsilon=0.03125, gamma=0.0010416666666666667))])


In [12]:
y_pred_svr = best_svr_model_t2.predict(X_test.values)

In [13]:
MAE_SVR_t2=metrics.mean_absolute_error(y_test['percentage(t+2)'], y_pred_svr)
print('MAE en t+2: {:2.2%}'.format(MAE_SVR_t2))

MAE en t+2: 1.61%


In [14]:
joblib.dump(best_svr_model_t2, "modelos_exogena/SVR_t2_model.joblib")

['modelos_exogena/SVR_t2_model.joblib']

# t+2

In [21]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-7, -3)]) / 30)
l_epsilon = [2.**k for k in range(-6, 0)]

In [22]:
l_C

[10.0, 100.0, 1000.0, 10000.0, 100000.0]

In [23]:
l_gamma

[0.00026041666666666666,
 0.0005208333333333333,
 0.0010416666666666667,
 0.0020833333333333333]

In [24]:
l_epsilon

[0.015625, 0.03125, 0.0625, 0.125, 0.25, 0.5]

In [25]:
start = time()
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t2 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t2.fit(X_train.values, y_train['percentage(t+2)'].values)

training_time = time() - start
print(training_time/60, ' minutos')

24.352999126911165  minutos


In [26]:
best_svr_score_t2 = grid_search_SVM_t2.best_score_
best_svr_model_t2 = grid_search_SVM_t2.best_estimator_

In [27]:
best_svr_model_t2.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=10.0, epsilon=0.03125, gamma=0.0020833333333333333))])

In [28]:
print(best_svr_model_t2.regressor_)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=10.0, epsilon=0.03125, gamma=0.0020833333333333333))])


In [29]:
y_pred_svr = best_svr_model_t2.predict(X_test.values)

In [30]:
MAE_SVR_t2=metrics.mean_absolute_error(y_test['percentage(t+2)'], y_pred_svr)
print('MAE en t+2: {:2.2%}'.format(MAE_SVR_t2))

MAE en t+2: 1.76%


In [20]:
# joblib.dump(best_svr_model_t2, "modelos_exogena/SVR_t2_model.joblib")

['modelos_exogena/SVR_t2_model.joblib']

# t+3

In [4]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 30)
l_epsilon = [2.**k for k in range(-6, -2)]

In [5]:
l_gamma

[0.0005208333333333333,
 0.0010416666666666667,
 0.0020833333333333333,
 0.004166666666666667]

In [6]:
l_C

[10.0, 100.0, 1000.0, 10000.0, 100000.0]

In [7]:
l_epsilon

[0.015625, 0.03125, 0.0625, 0.125]

In [8]:
start = time()
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t3 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t3.fit(X_train.values, y_train['percentage(t+3)'].values)

training_time = time() - start
print(training_time/60, ' minutos')

37.53236649831136  minutos


In [9]:
best_svr_score = grid_search_SVM_t3.best_score_
best_svr_model = grid_search_SVM_t3.best_estimator_

In [10]:
best_svr_model.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=100.0, epsilon=0.125, gamma=0.0010416666666666667))])

In [11]:
y_pred_svr = best_svr_model.predict(X_test.values)

In [12]:
MAE_SVR_t3=metrics.mean_absolute_error(y_test['percentage(t+3)'], y_pred_svr)
print('MAE en t+3: {:2.2%}'.format(MAE_SVR_t3))

MAE en t+3: 2.24%


In [13]:
joblib.dump(best_svr_model, "modelos_exogena/SVR_t3_model.joblib")

['modelos_exogena/SVR_t3_model.joblib']

# t+4

In [4]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 30)
l_epsilon = [2.**k for k in range(-6, -2)]

In [5]:
start = time()
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t4 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t4.fit(X_train.values, y_train['percentage(t+4)'].values)

training_time = time() - start
print(training_time/60, ' minutos')

34.841022566954294  minutos


In [6]:
best_svr_score_t4 = grid_search_SVM_t4.best_score_
best_svr_model_t4 = grid_search_SVM_t4.best_estimator_

In [7]:
best_svr_model_t4.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=100.0, epsilon=0.0625, gamma=0.0005208333333333333))])

In [8]:
y_pred_svr = best_svr_model_t4.predict(X_test.values)

In [9]:
MAE_SVR_t4=metrics.mean_absolute_error(y_test['percentage(t+4)'], y_pred_svr)
print('MAE en t+4: {:2.2%}'.format(MAE_SVR_t4))

MAE en t+4: 2.86%


In [10]:
joblib.dump(best_svr_model_t4, "modelos_exogena/SVR_t4_model.joblib")

['modelos_exogena/SVR_t4_model.joblib']

# t+5

In [4]:
# l_C = [10.**k for k in range(2, 8)]
# l_gamma = list( np.array([2.**k for k in range(-8, -4)]) / 12)
# l_epsilon = [2.**k for k in range(-6, -2)]

In [4]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 30)
l_epsilon = [2.**k for k in range(-6, -2)]

In [5]:
print('C: ', l_C)
print('gamma:', l_gamma)
print('epsilon:', l_epsilon)

C:  [10.0, 100.0, 1000.0, 10000.0, 100000.0]
gamma: [0.0005208333333333333, 0.0010416666666666667, 0.0020833333333333333, 0.004166666666666667]
epsilon: [0.015625, 0.03125, 0.0625, 0.125]


In [6]:
start = time()
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t5 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t5.fit(X_train.values, y_train['percentage(t+5)'].values)

training_time = time() - start
print(training_time/60, ' minutos')

32.280841950575514  minutos


In [7]:
best_svr_score_t5 = grid_search_SVM_t5.best_score_
best_svr_model_t5 = grid_search_SVM_t5.best_estimator_

In [8]:
best_svr_model_t5.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=100.0, epsilon=0.0625, gamma=0.0010416666666666667))])

In [9]:
y_pred_svr = best_svr_model_t5.predict(X_test.values)

In [10]:
MAE_SVR_t5=metrics.mean_absolute_error(y_test['percentage(t+5)'], y_pred_svr)
print('MAE en t+5: {:2.2%}'.format(MAE_SVR_t5))

MAE en t+5: 3.24%


In [12]:
# joblib.dump(best_svr_model_t5, "modelos_exogena/SVR_t5_model.joblib")

['modelos_exogena/SVR_t5_model.joblib']

In [11]:
# files.download("SVR_t5_model.joblib")

# t+6

In [5]:
# l_C = [10.**k for k in range(2, 3)]
# l_gamma = list( np.array([2.**k for k in range(-7, -4)]) / 12)
# l_epsilon = [2.**k for k in range(-4, -1)]

In [4]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 12)
l_epsilon = [2.**k for k in range(-6, -2)]

In [5]:
l_C

[10.0, 100.0, 1000.0, 10000.0, 100000.0]

In [6]:
l_gamma

[0.0013020833333333333,
 0.0026041666666666665,
 0.005208333333333333,
 0.010416666666666666]

In [7]:
l_epsilon

[0.015625, 0.03125, 0.0625, 0.125]

In [8]:
start = time()
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t6 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t6.fit(X_train.values, y_train['percentage(t+6)'].values)

training_time = time() - start
print(training_time/60, ' minutos')

68.50439644257227  minutos


In [9]:
# !pip list

In [10]:
best_svr_score_t6 = grid_search_SVM_t6.best_score_
best_svr_model_t6 = grid_search_SVM_t6.best_estimator_

In [11]:
best_svr_model_t6.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=10.0, epsilon=0.03125, gamma=0.0026041666666666665))])

In [12]:
y_pred_svr = best_svr_model_t6.predict(X_test.values)

In [13]:
MAE_SVR_t6=metrics.mean_absolute_error(y_test['percentage(t+6)'], y_pred_svr)
print('MAE en t+6: {:2.2%}'.format(MAE_SVR_t6))

MAE en t+6: 3.38%


In [14]:
joblib.dump(best_svr_model_t6, "modelos_exogena/SVR_t6_model.joblib")

['modelos_exogena/SVR_t6_model.joblib']

In [ ]:
# files.download("SVR_t6_model.joblib")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>